###画像の前処理
* KaggleからCubiCasa5kダウンロード
* いただいたスクリプトでマスク画像生成
* 元画像としてsvg→png変換
- 512×512にリサイズ
- マスクが複数のフロアがある場合取得できていない（display=Noneをスルーしなくするように変更）
- マスクとオリジナル画像のサイズと位置がずれていた（左上を基準にcropすることで一致）


###訓練中
- 格子を全画像に追加
- 格子間隔はランダム
- 格子を点線化
- largeモデルを使用
- cubicasaのHigh quality architecturalを使用

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install -q datasets albumentations

### 必要に応じて実行

In [ ]:
!unzip -q resize_without_noise.zip

In [ ]:
!unzip -q /content/drive/MyDrive/models/homes_2.zip -d /content/drive/MyDrive/models/

In [ ]:
rm -r processed_data

In [ ]:
!unzip -q processed_data_grid.zip -d grid

In [ ]:
!unzip -qo /content/drive/MyDrive/models/resize_architectural.zip -d ./

In [ ]:
!unzip -qo /content/drive/MyDrive/models/processed_data_high_512_512.zip -d ./

In [ ]:
!zip -rq /content/drive/MyDrive/models/processed_data_high_512_512.zip ./processed_data

In [ ]:
!cp -r processed_data /content/drive/MyDrive/models/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mv processed_data/annotations/validation/* processed_data/annotations/train/
!mv processed_data/annotations/test/* processed_data/annotations/train/
!mv processed_data/images/validation/* processed_data/images/train/
!mv processed_data/images/test/* processed_data/images/train/

In [ ]:
!mv grid/processed_data/images/test/* processed_data/images/test/
!mv grid/processed_data/images/validation/* processed_data/images/validation/
!mv grid/processed_data/annotations/test/* processed_data/annotations/test/
!mv grid/processed_data/annotations/validation/* processed_data/annotations/validation/

In [ ]:
!rm processed_data/annotations/train/2ldk_5.png
!cp processed_data/annotations/train/original_floorplan_1268.png processed_data/annotations/test/
!mv processed_data/annotations/train/original_floorplan_1268.png processed_data/annotations/validation/
!rm processed_data/images/train/2ldk_5.png
!cp processed_data/images/train/original_floorplan_1268.png processed_data/images/test/
!mv processed_data/images/train/original_floorplan_1268.png processed_data/images/validation/

In [ ]:
import os
from PIL import Image
from tqdm import tqdm

input_dir = "./mask2former_resize2/"  # 元画像があるディレクトリ
output_dir = "./mask2former_resize_resized2"  # リサイズ後の画像を保存するディレクトリ

os.makedirs(output_dir, exist_ok=True)

resize_width = 512
resize_height = 512

# 指定ディレクトリ内の画像をリサイズして保存
for file_name in tqdm(os.listdir(input_dir)):
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
        image_path = os.path.join(input_dir, file_name)
        image = Image.open(image_path).convert("RGB")

        # リサイズ
        image_resized = image.resize((resize_width, resize_height), Image.NEAREST)

        # リサイズ後の画像を保存
        save_path = os.path.join(output_dir, file_name)
        image_resized.save(save_path)


In [ ]:
import os
import json
from PIL import Image
import numpy as np
import torch
import random
import cv2
from datasets import Dataset, DatasetDict
from tqdm import tqdm

# 入力ディレクトリと出力ディレクトリの設定
# ここでは "original_floorplan.png" と "room_mask_color.png" があるフォルダを指定
input_dir = "./mask2former_resize_resized/"
output_dir = "./processed_data"

os.makedirs(f"{output_dir}/images/train", exist_ok=True)
os.makedirs(f"{output_dir}/images/validation", exist_ok=True)
os.makedirs(f"{output_dir}/images/test", exist_ok=True)
os.makedirs(f"{output_dir}/annotations/train", exist_ok=True)
os.makedirs(f"{output_dir}/annotations/validation", exist_ok=True)
os.makedirs(f"{output_dir}/annotations/test", exist_ok=True)

# id_to_label辞書: クラスIDを定義 (ここでは 1: "room" のみ)
id_to_label = {
    1: "room",
}
print("Generated id_to_label mapping:", id_to_label)


In [ ]:
# For cubicasa5k

files = [(f"original_floorplan_{i}.png", f"room_mask_color_{i}.png") for i in range(1269)]

# シャッフル
# random.shuffle(files)


train_files = files[0:]


# カラーインスタンスマスクを読み込んで、annotation_mask に変換する処理
def convert_color_mask_to_annotation(color_mask_bgr):
    """
    color_mask_bgr: OpenCVで読み込んだ BGR配列 (height, width, 3)
    戻り値: annotation_mask (同じサイズのBGRまたはRGB配列)
    """
    # アウトプット用
    height, width, _ = color_mask_bgr.shape
    annotation_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # BGR -> unique colors
    # まずユニークな色を取得
    #  shapeを (H*W,3) に潰してからnp.unique() すればOK
    unique_colors = np.unique(color_mask_bgr.reshape(-1, 3), axis=0)

    instance_id = 0
    for c in unique_colors:
        # c は (B,G,R) の1ピクセルカラー
        # 背景(0,0,0)はスキップ
        if np.all(c == [0,0,0]):
            continue

        # インスタンスIDを1増やす
        instance_id += 1

        # c と一致するピクセルを抽出
        mask = np.all(color_mask_bgr == c, axis=-1)  # shape=(H,W), bool

        # アノテーション上では [0, クラスID=1, インスタンスID] という3chに書き込む
        annotation_mask[mask] = [0, instance_id, 1]

    return annotation_mask

# 読み込んで、train/validation/test に分けて保存する関数
def process_floorplan_and_mask(floorplan_png, mask_png, split):
    """
    floorplan_png: 元画像(フロア図)のパス
    mask_png: 部屋マスク(カラーインスタンス)のパス
    split: "train"/"validation"/"test"
    """

    # 画像を読み込み
    floorplan_bgr = cv2.imread(floorplan_png, cv2.IMREAD_COLOR)  # BGR
    color_mask_bgr = cv2.imread(mask_png, cv2.IMREAD_COLOR)      # BGR

    if floorplan_bgr is None:
        print(f"Error: cannot read {floorplan_png}")
        return
    if color_mask_bgr is None:
        print(f"Error: cannot read {mask_png}")
        return

    # アノテーションマスクを生成
    annotation_mask = convert_color_mask_to_annotation(color_mask_bgr)

    # 出力ファイル名
    base_name = os.path.splitext(os.path.basename(floorplan_png))[0]
    image_id = f"{base_name}"

    # 保存先
    image_save_path = f"{output_dir}/images/{split}/{image_id}.jpg"
    annotation_save_path = f"{output_dir}/annotations/{split}/{image_id}.png"

    # OpenCVで書き出し
    cv2.imwrite(image_save_path, floorplan_bgr)

    # アノテーションを保存 (3ch PNG)
    cv2.imwrite(annotation_save_path, annotation_mask)


# 分割ごとに処理
for split, file_list in [("train", train_files),]:
    for (floorplan_png, mask_png) in tqdm(file_list):
        process_floorplan_and_mask(os.path.join(input_dir, floorplan_png),
                                   os.path.join(input_dir, mask_png),
                                   split)

In [ ]:
# For cubicasa5k_high
input_dir = "./mask2former_resize_resized2/"
files = [(f"white_original_floorplan_{i}.png", f"room_mask_color_{i}.png") for i in range(4000)]

# シャッフル
# random.shuffle(files)


train_files = files[0:]


# カラーインスタンスマスクを読み込んで、annotation_mask に変換する処理
def convert_color_mask_to_annotation(color_mask_bgr):
    """
    color_mask_bgr: OpenCVで読み込んだ BGR配列 (height, width, 3)
    戻り値: annotation_mask (同じサイズのBGRまたはRGB配列)
    """
    # アウトプット用
    height, width, _ = color_mask_bgr.shape
    annotation_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # BGR -> unique colors
    # まずユニークな色を取得
    #  shapeを (H*W,3) に潰してからnp.unique() すればOK
    unique_colors = np.unique(color_mask_bgr.reshape(-1, 3), axis=0)

    instance_id = 0
    for c in unique_colors:
        # c は (B,G,R) の1ピクセルカラー
        # 背景(0,0,0)はスキップ
        if np.all(c == [0,0,0]):
            continue

        # インスタンスIDを1増やす
        instance_id += 1

        # c と一致するピクセルを抽出
        mask = np.all(color_mask_bgr == c, axis=-1)  # shape=(H,W), bool

        # アノテーション上では [0, クラスID=1, インスタンスID] という3chに書き込む
        annotation_mask[mask] = [0, instance_id, 1]

    return annotation_mask

# 読み込んで、train/validation/test に分けて保存する関数
def process_floorplan_and_mask(floorplan_png, mask_png, split):
    """
    floorplan_png: 元画像(フロア図)のパス
    mask_png: 部屋マスク(カラーインスタンス)のパス
    split: "train"/"validation"/"test"
    """

    # 画像を読み込み
    floorplan_bgr = cv2.imread(floorplan_png, cv2.IMREAD_COLOR)  # BGR
    color_mask_bgr = cv2.imread(mask_png, cv2.IMREAD_COLOR)      # BGR

    if floorplan_bgr is None:
        print(f"Error: cannot read {floorplan_png}")
        return
    if color_mask_bgr is None:
        print(f"Error: cannot read {mask_png}")
        return

    # アノテーションマスクを生成
    annotation_mask = convert_color_mask_to_annotation(color_mask_bgr)

    # 出力ファイル名
    base_name = os.path.splitext(os.path.basename(floorplan_png))[0]
    image_id = f"{base_name}_high"

    # 保存先
    image_save_path = f"{output_dir}/images/{split}/{image_id}.jpg"
    annotation_save_path = f"{output_dir}/annotations/{split}/{image_id}.png"

    # OpenCVで書き出し
    cv2.imwrite(image_save_path, floorplan_bgr)

    # アノテーションを保存 (3ch PNG)
    cv2.imwrite(annotation_save_path, annotation_mask)


# 分割ごとに処理
for split, file_list in [("train", train_files),]:
    for (floorplan_png, mask_png) in tqdm(file_list):
        process_floorplan_and_mask(os.path.join(input_dir, floorplan_png),
                                   os.path.join(input_dir, mask_png),
                                   split)

In [ ]:
# For homes

input_dir = "/content/drive/MyDrive/models/homes"  # Labelme形式のJSONと対応する画像が保存されたディレクトリ
output_dir = "./processed_data"  # 出力フォルダ

# JSONファイルをリスト化
json_files = [f for f in os.listdir(input_dir) if f.endswith(".json")]

# ファイルをシャッフル
#random.shuffle(json_files)

# 分割
train_files = json_files[0:]

# id_to_label辞書を作成
id_to_label = {
    1:"room",
}

print("Generated id_to_label mapping:")
print(id_to_label)

# JSONファイルの処理関数
def process_json_file(json_path, split):
    with open(json_path, "r") as f:
        data = json.load(f)

    # 対応する画像ファイルをロード
    image_path = os.path.join(input_dir, data["imagePath"])
    image = Image.open(image_path).convert("RGB")

    # アノテーションマスクを生成
    annotation_mask = np.zeros((image.height, image.width, 3), dtype=np.uint8)


    instance_id = 0
    for shape in data["shapes"]:
        instance_id += 1
        points = shape["points"]

        # ポリゴンを塗りつぶす
        points = np.array(points, dtype=np.int32)
        fill_color = (1, instance_id, 0)
        cv2.fillPoly(annotation_mask, [points], fill_color)

    # 保存用のIDを作成
    image_id = os.path.splitext(os.path.basename(json_path))[0]

    # 画像とアノテーションを保存
    image.save(f"{output_dir}/images/{split}/{image_id}.jpg")
    annotation = Image.fromarray(annotation_mask)
    annotation.save(f"{output_dir}/annotations/{split}/{image_id}.png")

# データセットの分割ごとに処理
for split, files in [("train", train_files),]:
    for json_file in files:
        json_path = os.path.join(input_dir, json_file)
        process_json_file(json_path, split)

In [ ]:
# For valdata

input_dir = "/content/drive/MyDrive/models/valdata"  # Labelme形式のJSONと対応する画像が保存されたディレクトリ
output_dir = "./processed_data"  # 出力フォルダ

# JSONファイルをリスト化
json_files = [f for f in os.listdir(input_dir) if f.endswith(".json")]

# ファイルをシャッフル
#random.shuffle(json_files)

# 分割
validation_files = json_files[0:]
test_files = json_files[0:]

# id_to_label辞書を作成
id_to_label = {
    1:"room",
}

print("Generated id_to_label mapping:")
print(id_to_label)

# JSONファイルの処理関数
def process_json_file(json_path, split):
    with open(json_path, "r") as f:
        data = json.load(f)

    # 対応する画像ファイルをロード
    image_path = os.path.join(input_dir, data["imagePath"])
    image = Image.open(image_path).convert("RGB")

    # アノテーションマスクを生成
    annotation_mask = np.zeros((image.height, image.width, 3), dtype=np.uint8)


    instance_id = 0
    for shape in data["shapes"]:
        instance_id += 1
        points = shape["points"]

        # ポリゴンを塗りつぶす
        points = np.array(points, dtype=np.int32)
        fill_color = (1, instance_id, 0)
        cv2.fillPoly(annotation_mask, [points], fill_color)

    # 保存用のIDを作成
    image_id = os.path.splitext(os.path.basename(json_path))[0]

    # 画像とアノテーションを保存
    image.save(f"{output_dir}/images/{split}/{image_id}.jpg")
    annotation = Image.fromarray(annotation_mask)
    annotation.save(f"{output_dir}/annotations/{split}/{image_id}.png")

# データセットの分割ごとに処理
for split, files in [("validation", validation_files), ("test", test_files)]:
    for json_file in files:
        json_path = os.path.join(input_dir, json_file)
        process_json_file(json_path, split)

### ここからは共通の処理

In [ ]:
import os
import json
from PIL import Image
import numpy as np
import torch
import random
import cv2
from datasets import Dataset, DatasetDict
from tqdm import tqdm

output_dir = "./processed_data"

id_to_label = {
    1: "room",
}

# DatasetDictを作成
def load_data(split):
    images_dir = os.path.join(output_dir, f"images/{split}")
    annotations_dir = os.path.join(output_dir, f"annotations/{split}")

    images = []
    annotations = []

    for image_file in tqdm(os.listdir(images_dir)):
        image_path = os.path.join(images_dir, image_file)
        annotation_file = image_file.replace(".jpg", ".png")
        annotation_path = os.path.join(annotations_dir, annotation_file)

        # データをロード
        image = Image.open(image_path).convert("RGB")  # 画像をRGB形式に変換
        annotation = Image.open(annotation_path).convert("RGB")

        # データを追加
        images.append(image)
        annotations.append(annotation)

    # 学習データ(train)の場合のみシャッフルをかける
    if split == "train":
        # (画像, アノテーション) をペアにして1つのリストとしてまとめる
        combined = list(zip(images, annotations))
        # combined リストをシャッフル
        random.shuffle(combined)
        # 再度アンパックして images, annotations に戻す
        images, annotations = zip(*combined)
        # 必要に応じてリスト化しておく
        images, annotations = list(images), list(annotations)

    return {"image": images, "annotation": annotations}

# データセットをロードしてDatasetDictを作成
datasets = {}
for split in ["train", "validation", "test"]:
    data = load_data(split)
    datasets[split] = Dataset.from_dict(data)

dataset_dict = DatasetDict(datasets)

print(dataset_dict)

In [ ]:
import numpy as np

example = datasets['train'][1]
seg = np.array(example['annotation'])
# get green channel
np.set_printoptions(threshold=np.inf)


instance_seg = seg[:, :, 0]

In [ ]:
np.unique(instance_seg)

In [ ]:
instance_seg = np.array(example["annotation"])[:,:,1] # green channel encodes instances
class_id_map = np.array(example["annotation"])[:,:,0] # red channel encodes semantic category
class_labels = np.unique(class_id_map)

# create mapping between instance IDs and semantic category IDs
inst2class = {}
for label in class_labels:
    instance_ids = np.unique(instance_seg[class_id_map == label])
    inst2class.update({i: label for i in instance_ids})
print(inst2class)

In [ ]:
from transformers import Mask2FormerImageProcessor

processor = Mask2FormerImageProcessor(reduce_labels=True, ignore_index=255, do_resize=False, do_rescale=False, do_normalize=False)

In [ ]:
import numpy as np
from torch.utils.data import Dataset

class ImageSegmentationDataset(Dataset):
    """Image segmentation dataset."""

    def __init__(self, dataset, processor, transform=None):
        """
        Args:
            dataset
        """
        self.dataset = dataset
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = np.array(self.dataset[idx]["image"].convert("RGB"))

        instance_seg = np.array(self.dataset[idx]["annotation"])[:,:,1]
        class_id_map = np.array(self.dataset[idx]["annotation"])[:,:,0]
        class_labels = np.unique(class_id_map)

        inst2class = {}
        for label in class_labels:
            instance_ids = np.unique(instance_seg[class_id_map == label])
            inst2class.update({i: label for i in instance_ids})

        # apply transforms
        if self.transform is not None:
            transformed = self.transform(image=image, mask=instance_seg)
            image, instance_seg = transformed['image'], transformed['mask']
            # convert to C, H, W
            image = image.transpose(2,0,1)

        if class_labels.shape[0] == 1 and class_labels[0] == 0:
            # Some image does not have annotation (all ignored)
            inputs = self.processor([image], return_tensors="pt")
            inputs = {k:v.squeeze() for k,v in inputs.items()}
            inputs["class_labels"] = torch.tensor([0])
            inputs["mask_labels"] = torch.zeros((0, inputs["pixel_values"].shape[-2], inputs["pixel_values"].shape[-1]))
        else:
          inputs = self.processor([image], [instance_seg], instance_id_to_semantic_id=inst2class, return_tensors="pt")
          inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}

        return inputs

In [ ]:
import albumentations as A
import cv2
import random

class GridNoise(A.ImageOnlyTransform):
    """
    画像にランダムに格子状のノイズを入れるTransform。
    - p: このTransformを適用する確率
    - min_grid_spacing, max_grid_spacing: 格子の間隔をとる乱数の範囲
    - min_dash_length, max_dash_length: dash_length をとる乱数の範囲
    - dash_length, gap_length はそれぞれランダムに取りたいので
      get_paramsで決定する。
    - line_thickness: 線の太さ
    """
    def __init__(
        self,
        p=1.0,
        min_grid_spacing=20,
        max_grid_spacing=70,
        min_dash_length=1,
        max_dash_length=10,
        line_thickness=1,
        always_apply=False
    ):
        super().__init__(always_apply=always_apply, p=p)
        self.min_grid_spacing = min_grid_spacing
        self.max_grid_spacing = max_grid_spacing
        self.min_dash_length = min_dash_length
        self.max_dash_length = max_dash_length
        self.line_thickness = line_thickness

    def get_params(self):
        """
        Albumentations が変換をかけるたびに呼ばれ、
        apply() に渡すパラメータが生成される。
        """
        grid_spacing = random.randint(self.min_grid_spacing, self.max_grid_spacing)
        dash_length = random.randint(self.min_dash_length, self.max_dash_length)
        gap_length = random.randint(self.min_dash_length, self.max_dash_length)
        return {
            "grid_spacing": grid_spacing,
            "dash_length": dash_length,
            "gap_length": gap_length,
        }

    def apply(self, img, grid_spacing=50, dash_length=0, gap_length=0, **params):
        """
        実際に img に点線/格子を描画する。
        """
        height, width, _ = img.shape

        # 水平線を描画
        for y in range(0, height, grid_spacing):
            self._draw_dashed_line_horizontal(img, y, width, dash_length, gap_length)

        # 垂直線を描画
        for x in range(0, width, grid_spacing):
            self._draw_dashed_line_vertical(img, x, height, dash_length, gap_length)

        return img

    def _draw_dashed_line_horizontal(self, img, y, width, dash_length, gap_length):
        color = (0, 0, 0)  # 黒
        if dash_length <= 0 or gap_length <= 0:
            # dash_length, gap_length が 0 以下なら実線
            cv2.line(img, (0, y), (width - 1, y), color, self.line_thickness)
        else:
            x = 0
            while x < width:
                x_end = min(x + dash_length, width - 1)
                cv2.line(img, (x, y), (x_end, y), color, self.line_thickness)
                x = x_end + gap_length

    def _draw_dashed_line_vertical(self, img, x, height, dash_length, gap_length):
        color = (0, 0, 0)  # 黒
        if dash_length <= 0 or gap_length <= 0:
            # dash_length, gap_length が 0 以下なら実線
            cv2.line(img, (x, 0), (x, height - 1), color, self.line_thickness)
        else:
            y = 0
            while y < height:
                y_end = min(y + dash_length, height - 1)
                cv2.line(img, (x, y), (x, y_end), color, self.line_thickness)
                y = y_end + gap_length

# 使い方の例
# p=0.5, grid_spacing=50, line_thickness=1
# dash_length=10, gap_length=5 なら、
# 「線長さ10ピクセル、隙間5ピクセルの破線」を描画する
train_transform = A.Compose([
    A.Resize(width=512, height=512),
    GridNoise(
        p=1.0,
        min_grid_spacing=20,
        max_grid_spacing=70,
        min_dash_length=1,
        max_dash_length=10,
        line_thickness=1
    ),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# バリデーション・テストはノイズなし
val_transform = A.Compose([
    A.Resize(width=512, height=512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
], is_check_shapes=False)

train_dataset = ImageSegmentationDataset(datasets["train"], processor=processor, transform=train_transform)
val_dataset = ImageSegmentationDataset(datasets["validation"], processor=processor, transform=val_transform)

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    pixel_values = torch.stack([example["pixel_values"] for example in batch])
    pixel_mask = torch.stack([example["pixel_mask"] for example in batch])
    class_labels = [example["class_labels"] for example in batch]
    mask_labels = [example["mask_labels"] for example in batch]
    return {"pixel_values": pixel_values, "pixel_mask": pixel_mask, "class_labels": class_labels, "mask_labels": mask_labels}

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation

# Replace the head of the pre-trained model
# We specify ignore_mismatched_sizes=True to replace the already fine-tuned classification head by a new one
#facebook/mask2former-swin-large-coco-instance
#facebook/mask2former-swin-large-ade-panoptic
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-instance",
                                                          id2label=id_to_label,
                                                          ignore_mismatched_sizes=True)


In [ ]:
# すべてのクラスラベルを1つのマスクに統合する関数
def combine_all_classes(results):
    """
    すべてのセグメントを1つのマスクに統合する。
    """
    segmentation = results['segmentation'].cpu().numpy()
    unified_mask = np.zeros_like(segmentation, dtype=bool)

    # セグメントを統合
    for segment in results['segments_info']:
        segment_mask = (segmentation == segment['id'])
        unified_mask |= segment_mask  # 全体のマスクに統合

    # 統合マスクを画像形式に変換
    unified_mask_image = Image.fromarray((unified_mask * 255).astype(np.uint8))

    return unified_mask_image

In [ ]:
example = datasets['test'][0]
image_test1 = example['image']
example = datasets['test'][1]
image_test2 = example['image']
display(image_test1)
display(image_test2)


In [ ]:
import torch
from tqdm.auto import tqdm
from transformers import MaskFormerImageProcessor

processor_test = MaskFormerImageProcessor()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

running_loss = 0.0
num_samples = 0
best_val_loss = float("inf")
best_model_path = "mask2former_best"
num_epochs=10

for epoch in range(num_epochs):
  print("Epoch:", epoch)
  model.train()

  for idx, batch in enumerate(tqdm(train_dataloader)):
      # Reset the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(
              pixel_values=batch["pixel_values"].to(device),
              mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
              class_labels=[labels.to(device) for labels in batch["class_labels"]],
      )

      # Backward propagation
      loss = outputs.loss
      loss.backward()

      batch_size = batch["pixel_values"].size(0)
      running_loss += loss.item()
      num_samples += batch_size

      if idx % 300 == 0:
        print("Loss:", running_loss/num_samples)

      # Optimization
      optimizer.step()

  # ==== Validation ====
  model.eval()
  val_loss = 0.0
  val_samples = 0

  with torch.no_grad():
      for val_batch in tqdm(val_dataloader, desc="Validation"):
          outputs_val = model(
              pixel_values=val_batch["pixel_values"].to(device),
              mask_labels=[labels.to(device) for labels in val_batch["mask_labels"]],
              class_labels=[labels.to(device) for labels in val_batch["class_labels"]],
          )

          loss_val = outputs_val.loss

          batch_size_val = val_batch["pixel_values"].size(0)
          val_loss += loss_val.item() * batch_size_val
          val_samples += batch_size_val

  epoch_val_loss = val_loss / val_samples
  print(f"[Epoch {epoch}] Validation Loss: {epoch_val_loss:.4f}")

  # best_val_lossを更新した場合のみモデルを保存
  if epoch_val_loss < best_val_loss:
      best_val_loss = epoch_val_loss
      print(f"=> New best model found! Saving model (val_loss: {best_val_loss:.4f})")
      model.save_pretrained(best_model_path)
      processor.save_pretrained(best_model_path)

  # 出力してみる
  inputs_test1 = processor_test(image_test1, return_tensors="pt").to(device)
  inputs_test2 = processor_test(image_test2, return_tensors="pt").to(device)
  with torch.no_grad():
    outputs_test1 = model(**inputs_test1)
    outputs_test2 = model(**inputs_test2)
  results_test1 = processor_test.post_process_instance_segmentation(outputs_test1, target_sizes=[image_test1.size[::-1]])[0]
  results_test2 = processor_test.post_process_instance_segmentation(outputs_test2, target_sizes=[image_test2.size[::-1]])[0]
  # 全体のマスクを生成
  unified_mask = combine_all_classes(results_test1)
  display(unified_mask)
  unified_mask = combine_all_classes(results_test2)
  display(unified_mask)


In [ ]:
!cp -r mask2former_best /content/drive/MyDrive/models/large_high

In [ ]:
from google.colab import runtime

runtime.unassign()